# Examples with Partitions

> **These are Go notebooks**: In order to access the GoNB Jupyter Kernel, please install GoNB from here: [https://github.com/janpfeifer/gonb](https://github.com/janpfeifer/gonb)

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/stochadex=/path/to/stochadex"` at the top of any cell.

## UNIX timestamps

Running the simulator with UNIX timestamps representing the time variable makes plotting correctly formatted date-time strings possible with `go-echarts`. Here's a complete example using some simple Wiener processes below.

In [ ]:
import (
	"strconv"
	
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// How do you create a simulator.StateTimeStorage from a simulation run?
storage := analysis.NewStateTimeStorageFromPartitions(
	// What simulation state partitions do you want to create?
	[]*simulator.PartitionConfig{{
		Name:              "test_partition",
		Iteration:         &continuous.WienerProcessIteration{},
		Params:            simulator.NewParams(map[string][]float64{
			"variances": {1.0, 2.0, 3.0, 4.0},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              12345,
	}},
	// When should we stop the simulation?
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	// How should time evolve?
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	// What is the initial point in time?
	1667980544.0,
)

// What data would you like to plot on the x-axis?
xRef := analysis.PlotDataRef{
	PartitionName: "test_partition",
	IsTime: true,
}

// What data would you like to plot on the y-axis?
yRefs := make([]analysis.PlotDataRef, 0)
for i := 0; i < 4; i++ {
	yRefs = append(yRefs, analysis.PlotDataRef{
		PartitionName: "test_partition",
		ValueIndex: i,
	})
}

// How do you create a scatter plot from partitions in a simulator.StateTimeStorage? 
scatter := analysis.NewScatterPlotFromPartition(
	storage,
	xRef,
	yRefs,
)

// How do you display date-time strings when the time is a UNIX timestamp?
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// How do you display the plot in a Go notebook?
gonb_echarts.Display(
	scatter, 
	"width: 1024px; height:400px; background: white;",
)

## Custom iterations

WIP

In [ ]:
import (
	"strconv"
	
	"github.com/umbralcalc/stochadex/pkg/simulator"
	"github.com/umbralcalc/stochadex/pkg/analysis"
	"github.com/umbralcalc/stochadex/pkg/continuous"

	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/charts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

// How do a I create a new partition iteration struct?
type MyCustomIteration struct {
}

// What parameters and settings do you need to configure this iteration?
func (m *MyCustomIteration) Configure(
	partitionIndex int,
	settings *simulator.Settings,
) {
}

// How does this iteration actually change the state of the partition over time?
func (m *MyCustomIteration) Iterate(
	params *simulator.Params,
	partitionIndex int,
	stateHistories []*simulator.StateHistory,
	timestepsHistory *simulator.CumulativeTimestepsHistory,
) []float64 {
	return stateHistories[partitionIndex].Values.RawRowView(0)
}

%%

// How do you create a simulator.StateTimeStorage from a simulation run?
storage := analysis.NewStateTimeStorageFromPartitions(
	// What simulation state partitions do you want to create?
	[]*simulator.PartitionConfig{{
		Name:              "custom_partition",
		Iteration:         &MyCustomIteration{},
		Params:            simulator.NewParams(map[string][]float64{
			"variances": {1.0, 2.0, 3.0, 4.0},
		}),
		InitStateValues:   []float64{0.0, 0.0, 0.0, 0.0},
		StateHistoryDepth: 1,
		Seed:              12345,
	}},
	// When should we stop the simulation?
	&simulator.NumberOfStepsTerminationCondition{
		MaxNumberOfSteps: 100,
	},
	// How should time evolve?
	&simulator.ConstantTimestepFunction{
		Stepsize: 1000.0,
	},
	// What is the initial point in time?
	1667980544.0,
)

// What data would you like to plot on the x-axis?
xRef := analysis.PlotDataRef{
	PartitionName: "custom_partition",
	IsTime: true,
}

// What data would you like to plot on the y-axis?
yRefs := make([]analysis.PlotDataRef, 0)
for i := 0; i < 4; i++ {
	yRefs = append(yRefs, analysis.PlotDataRef{
		PartitionName: "custom_partition",
		ValueIndex: i,
	})
}

// How do you create a scatter plot from partitions in a simulator.StateTimeStorage? 
scatter := analysis.NewScatterPlotFromPartition(
	storage,
	xRef,
	yRefs,
)

// How do you display date-time strings when the time is a UNIX timestamp?
scatter.SetGlobalOptions(charts.WithXAxisOpts(opts.XAxis{Type: "time"}))

// How do you display the plot in a Go notebook?
gonb_echarts.Display(
	scatter, 
	"width: 1024px; height:400px; background: white;",
)